# Apache Spark Notebook


In [1]:
from pyspark import SparkContext, SparkConf, SQLContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import desc
import subprocess
from pyspark.sql.functions import col, max as max_

In [2]:
year = 'hdfs://localhost:9000/user/student/airline/1987.csv'

In [3]:
spark = SparkSession.builder.appName("performance-app").config("spark.config.option", "value").getOrCreate()

In [4]:
df = spark.read.option("header", "true").csv(year)

In [5]:
orig_airports = df.groupBy('Origin').count().orderBy(desc('count'))

In [6]:
orig_airports.first()

Row(Origin='ORD', count=67216)

In [7]:
df

DataFrame[Year: string, Month: string, DayofMonth: string, DayOfWeek: string, DepTime: string, CRSDepTime: string, ArrTime: string, CRSArrTime: string, UniqueCarrier: string, FlightNum: string, TailNum: string, ActualElapsedTime: string, CRSElapsedTime: string, AirTime: string, ArrDelay: string, DepDelay: string, Origin: string, Dest: string, Distance: string, TaxiIn: string, TaxiOut: string, Cancelled: string, CancellationCode: string, Diverted: string, CarrierDelay: string, WeatherDelay: string, NASDelay: string, SecurityDelay: string, LateAircraftDelay: string]

In [8]:
rdd = df.rdd

In [9]:
dir(rdd)

['__add__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getnewargs__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_computeFractionForSampleSize',
 '_defaultReducePartitions',
 '_id',
 '_is_barrier',
 '_jrdd',
 '_jrdd_deserializer',
 '_memory_limit',
 '_pickled',
 '_reserialize',
 '_to_java_object_rdd',
 'aggregate',
 'aggregateByKey',
 'barrier',
 'cache',
 'cartesian',
 'checkpoint',
 'coalesce',
 'cogroup',
 'collect',
 'collectAsMap',
 'combineByKey',
 'context',
 'count',
 'countApprox',
 'countApproxDistinct',
 'countByKey',
 'countByValue',
 'ctx',
 'distinct',
 'filter',
 'first',
 'flatMap',
 'flatMapValues',
 'fold',
 'foldByKey',
 'foreach',
 'foreachPartition',
 'fullOuterJoin',
 'getCheckpo

In [12]:
rdd.get(0)

AttributeError: 'RDD' object has no attribute 'get'

In [13]:
arr_delay_origins = df.withColumn('ArrDelay', col('ArrDelay').cast('integer')).groupBy('Origin').sum('ArrDelay')

In [14]:
dir(arr_delay_origins)


['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_collectAsArrow',
 '_jcols',
 '_jdf',
 '_jmap',
 '_jseq',
 '_lazy_rdd',
 '_repr_html_',
 '_sc',
 '_schema',
 '_sort_cols',
 '_support_repr_html',
 'agg',
 'alias',
 'approxQuantile',
 'cache',
 'checkpoint',
 'coalesce',
 'colRegex',
 'collect',
 'columns',
 'corr',
 'count',
 'cov',
 'createGlobalTempView',
 'createOrReplaceGlobalTempView',
 'createOrReplaceTempView',
 'createTempView',
 'crossJoin',
 'crosstab',
 'cube',
 'describe',
 'distinct',
 'drop',
 'dropDuplicates',
 'drop_duplicates',
 'dropna',
 'dtypes',
 'exceptAll',
 'explain',
 'fillna',
 'filter',
 'first',
 'foreach',
 'f

In [15]:
arr_delay_origins.orderBy(desc('sum(ArrDelay)')).show()

+------+-------------+
|Origin|sum(ArrDelay)|
+------+-------------+
|   LAX|       698222|
|   ATL|       608198|
|   SFO|       576020|
|   ORD|       536786|
|   DFW|       505833|
|   DEN|       473853|
|   PHX|       358504|
|   PIT|       312843|
|   DTW|       310199|
|   CLT|       302562|
|   BOS|       289699|
|   STL|       284874|
|   EWR|       281920|
|   MSP|       249633|
|   MEM|       214929|
|   PHL|       210120|
|   CVG|       206436|
|   SEA|       206139|
|   LGA|       206135|
|   SLC|       202789|
+------+-------------+
only showing top 20 rows



In [16]:
arr_delay_origins = df.withColumn('ArrDelay', col('ArrDelay').cast('integer'))\
                      .groupBy('Origin').sum('ArrDelay')\
                      .orderBy(desc('sum(ArrDelay)')).show()

+------+-------------+
|Origin|sum(ArrDelay)|
+------+-------------+
|   LAX|       698222|
|   ATL|       608198|
|   SFO|       576020|
|   ORD|       536786|
|   DFW|       505833|
|   DEN|       473853|
|   PHX|       358504|
|   PIT|       312843|
|   DTW|       310199|
|   CLT|       302562|
|   BOS|       289699|
|   STL|       284874|
|   EWR|       281920|
|   MSP|       249633|
|   MEM|       214929|
|   PHL|       210120|
|   CVG|       206436|
|   SEA|       206139|
|   LGA|       206135|
|   SLC|       202789|
+------+-------------+
only showing top 20 rows



In [17]:
most_arr_delay_origin = df.withColumn('ArrDelay', col('ArrDelay').cast('integer'))\
                          .groupBy('Origin').sum('ArrDelay')\
                          .orderBy(desc('sum(ArrDelay)')).first()

In [18]:
most_arr_delay_origin

Row(Origin='LAX', sum(ArrDelay)=698222)